# Regression Model

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

# -----------------------------
# Panel definition
# -----------------------------
stores = [f"S{i:02d}" for i in range(1, 11)]
weeks = pd.date_range("2023-01-02", "2025-12-29", freq="W-MON")

panel = pd.MultiIndex.from_product(
    [stores, weeks],
    names=["store_id", "week_start"]
).to_frame(index=False)

n = len(panel)

# -----------------------------
# Store-level latent effects
# -----------------------------
store_effect_gc = panel["store_id"].map(
    {s: np.random.normal(0, 0.15) for s in stores}
)

store_effect_ac = panel["store_id"].map(
    {s: np.random.normal(0, 0.10) for s in stores}
)

# -----------------------------
# Helper: generate correlated features
# -----------------------------
def generate_domain(prefix, n_features, base_scale=1.0):
    base = np.random.normal(0, base_scale, size=n)
    data = {}
    for i in range(1, n_features + 1):
        data[f"{prefix}_{i}"] = base + np.random.normal(0, 0.5, size=n)
    return pd.DataFrame(data)

# -----------------------------
# Feature domains (5 each)
# -----------------------------
digital_promo   = generate_domain("digital_promo", 5)
nondigital_promo = generate_domain("nondigital_promo", 5)
media            = generate_domain("media", 5)
menu             = generate_domain("menu", 5)
mix_share        = generate_domain("mix_share", 5)
csat             = generate_domain("csat", 5)
pricing          = generate_domain("pricing", 5)

X = pd.concat(
    [
        digital_promo,
        nondigital_promo,
        media,
        menu,
        mix_share,
        csat,
        pricing,
    ],
    axis=1,
)

# -----------------------------
# True elasticities (hidden)
# -----------------------------
beta_gc = {
    "digital_promo": 0.06,
    "nondigital_promo": 0.03,
    "media": 0.05,
    "menu": 0.04,
    "mix_share": 0.02,
    "csat": 0.03,
    "pricing": -0.08,
}

beta_ac = {
    "digital_promo": 0.01,
    "nondigital_promo": 0.015,
    "media": 0.005,
    "menu": 0.06,
    "mix_share": 0.04,
    "csat": 0.02,
    "pricing": 0.12,
}

def linear_combination(X, beta_map):
    out = np.zeros(n)
    for domain, beta in beta_map.items():
        cols = [c for c in X.columns if c.startswith(domain)]
        out += beta * X[cols].mean(axis=1)
    return out

# -----------------------------
# Generate targets (log space)
# -----------------------------
log_gc = (
    8.5
    + linear_combination(X, beta_gc)
    + store_effect_gc
    + np.random.normal(0, 0.15, size=n)
)

log_ac = (
    2.3
    + linear_combination(X, beta_ac)
    + store_effect_ac
    + np.random.normal(0, 0.08, size=n)
)

panel["guest_counts"] = np.exp(log_gc)
panel["average_check"] = np.exp(log_ac)

# Identity preserved
panel["sales"] = panel["guest_counts"] * panel["average_check"]

# -----------------------------
# Final dataset
# -----------------------------
df = pd.concat([panel, X], axis=1)

# Optional: add logs & deltas for diagnostics
df["log_gc"] = np.log(df["guest_counts"])
df["log_ac"] = np.log(df["average_check"])
df["log_sales"] = np.log(df["sales"])

df = df.sort_values(["store_id", "week_start"])
df["dlog_gc"] = df.groupby("store_id")["log_gc"].diff()
df["dlog_ac"] = df.groupby("store_id")["log_ac"].diff()
df["dlog_sales"] = df.groupby("store_id")["log_sales"].diff()

df.reset_index(drop=True, inplace=True)


In [3]:
df.head()

,store_id,week_start,guest_counts,average_check,sales,digital_promo_1,digital_promo_2,digital_promo_3,digital_promo_4,digital_promo_5,...,pricing_2,pricing_3,pricing_4,pricing_5,log_gc,log_ac,log_sales,dlog_gc,dlog_ac,dlog_sales
0,S01,2023-01-02,3954.187562,10.615962,41977.506316,0.929579,1.162397,0.871167,1.354849,2.229055,...,0.746199,-0.006621,0.941221,1.135517,8.282530,2.362359,10.644889,NaN,NaN,NaN
1,S01,2023-01-09,6265.926005,10.416270,65267.580205,-1.686452,0.002176,-0.250716,-0.946267,0.725503,...,-0.796966,-0.263924,-0.454098,-0.572339,8.742882,2.343369,11.086251,0.460351,-0.018990,0.441362
2,S01,2023-01-16,6016.031475,8.348131,50222.615937,0.285808,-0.162017,-0.455532,0.546936,1.500134,...,0.161102,-1.118125,0.025529,0.568790,8.702183,2.122038,10.824221,-0.040699,-0.221331,-0.262030
3,S01,2023-01-23,4401.770751,10.889535,47933.235085,-0.972781,-1.772048,-1.188044,-0.860905,-1.875745,...,1.531658,0.630777,0.365515,1.003285,8.389762,2.387802,10.777564,-0.312421,0.265765,-0.046656
4,S01,2023-01-30,5932.825495,9.177741,54449.936602,-1.725849,-1.121564,-0.398909,-1.518054,0.867727,...,-0.941251,-1.249387,-1.002577,-1.154580,8.688256,2.216781,10.905037,0.298494,-0.171021,0.127473


In [4]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd

def infer_feature_blocks(df: pd.DataFrame) -> Dict[str, List[str]]:
    prefixes = [
        "digital_promo", "nondigital_promo", "media", "menu",
        "mix_share", "csat", "pricing"
    ]
    blocks = {}
    for p in prefixes:
        blocks[p] = [c for c in df.columns if c.startswith(p + "_")]
    return blocks


In [5]:
def make_rolling_splits(
    df: pd.DataFrame,
    train_weeks: int = 104,
    test_weeks: int = 13,
    step_weeks: int = 13
) -> List[Tuple[pd.Timestamp, pd.Timestamp, pd.Timestamp, pd.Timestamp]]:
    weeks = sorted(df["week_start"].unique())
    splits = []
    for start_idx in range(0, len(weeks) - train_weeks - test_weeks + 1, step_weeks):
        train_start = weeks[start_idx]
        train_end   = weeks[start_idx + train_weeks - 1]
        test_start  = weeks[start_idx + train_weeks]
        test_end    = weeks[start_idx + train_weeks + test_weeks - 1]
        splits.append((train_start, train_end, test_start, test_end))
    return splits

def make_yoy_holdout_splits(
    df: pd.DataFrame,
    train_end_min: str = "2024-12-30",
    test_weeks: int = 13,
    step_weeks: int = 13
) -> List[Tuple[pd.Timestamp, pd.Timestamp, pd.Timestamp, pd.Timestamp]]:
    weeks = sorted(df["week_start"].unique())
    min_end = pd.Timestamp(train_end_min)
    splits = []
    for train_end in weeks:
        if train_end < min_end:
            continue
        test_start = train_end + pd.Timedelta(weeks=52)
        test_end   = test_start + pd.Timedelta(weeks=test_weeks - 1)
        if test_end > weeks[-1]:
            break
        train_start = weeks[0]
        splits.append((train_start, train_end, test_start, test_end))
    # thin by step
    return splits[::max(1, step_weeks // 13)]


In [6]:
def within_transform(
    df_sub: pd.DataFrame,
    y_col: str,
    x_cols: List[str],
    entity_col: str = "store_id"
) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Demean y and X within entity (store).
    Returns numpy arrays ready for sklearn.
    """
    g = df_sub.groupby(entity_col)
    y = df_sub[y_col] - g[y_col].transform("mean")
    X = df_sub[x_cols].copy()
    for c in x_cols:
        X[c] = X[c] - g[c].transform("mean")
    return y.values, X.values, x_cols


In [7]:
def add_mundlak_terms(
    df_sub: pd.DataFrame,
    x_cols: List[str],
    entity_col: str = "store_id",
    suffix: str = "_mean_store"
) -> Tuple[pd.DataFrame, List[str]]:
    g = df_sub.groupby(entity_col)
    mundlak = {}
    for c in x_cols:
        mundlak[c + suffix] = g[c].transform("mean")
    df_aug = df_sub.copy()
    for k, v in mundlak.items():
        df_aug[k] = v
    x_aug_cols = x_cols + list(mundlak.keys())
    return df_aug, x_aug_cols


In [8]:
import statsmodels.api as sm
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

@dataclass
class FitResult:
    coef: pd.Series
    se: pd.Series
    pval: pd.Series
    metrics: Dict[str, float]

def fit_ols_sm(y: np.ndarray, X: np.ndarray, colnames: List[str]) -> FitResult:
    Xc = sm.add_constant(X, has_constant="add")
    model = sm.OLS(y, Xc).fit()
    params = pd.Series(model.params[1:], index=colnames)  # drop const
    se = pd.Series(model.bse[1:], index=colnames)
    pval = pd.Series(model.pvalues[1:], index=colnames)

    yhat = model.predict(Xc)
    metrics = {
        "r2": float(model.rsquared),
        "rmse": float(np.sqrt(mean_squared_error(y, yhat))),
        "mae": float(mean_absolute_error(y, yhat)),
    }
    return FitResult(params, se, pval, metrics)

def fit_ridge(y: np.ndarray, X: np.ndarray, colnames: List[str], alpha: float) -> FitResult:
    model = Ridge(alpha=alpha, fit_intercept=True, random_state=42)
    model.fit(X, y)
    yhat = model.predict(X)
    coef = pd.Series(model.coef_, index=colnames)
    se = pd.Series(np.nan, index=colnames)
    pval = pd.Series(np.nan, index=colnames)
    metrics = {
        "r2": float(r2_score(y, yhat)),
        "rmse": float(np.sqrt(mean_squared_error(y, yhat))),
        "mae": float(mean_absolute_error(y, yhat)),
    }
    return FitResult(coef, se, pval, metrics)

def fit_elasticnet(y: np.ndarray, X: np.ndarray, colnames: List[str], alpha: float, l1_ratio: float) -> FitResult:
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True, random_state=42, max_iter=10000)
    model.fit(X, y)
    yhat = model.predict(X)
    coef = pd.Series(model.coef_, index=colnames)
    se = pd.Series(np.nan, index=colnames)
    pval = pd.Series(np.nan, index=colnames)
    metrics = {
        "r2": float(r2_score(y, yhat)),
        "rmse": float(np.sqrt(mean_squared_error(y, yhat))),
        "mae": float(mean_absolute_error(y, yhat)),
    }
    return FitResult(coef, se, pval, metrics)


In [ ]:
import json
import mlflow

def compute_coef_table(fit: FitResult) -> pd.DataFrame:
    dfc = pd.DataFrame({
        "feature": fit.coef.index,
        "coef": fit.coef.values,
        "se": fit.se.values,
        "pval": fit.pval.values,
    })
    dfc["sign"] = np.sign(dfc["coef"]).astype(int)
    dfc["abs_coef"] = np.abs(dfc["coef"])
    dfc["rank_abs"] = dfc["abs_coef"].rank(ascending=False, method="average")
    # "selected" for regularization: non-zero coefficient (or abs > eps)
    eps = 1e-10
    dfc["selected_flag"] = (dfc["abs_coef"] > eps).astype(int)
    return dfc.sort_values("abs_coef", ascending=False)

def run_one_experiment(
    df: pd.DataFrame,
    y_col: str,
    feature_cols: List[str],
    panel_method: str,          # "fe" or "mundlak"
    algorithm: str,             # "ols" | "ridge" | "elasticnet"
    algo_params: Dict,
    split: Tuple[pd.Timestamp, pd.Timestamp, pd.Timestamp, pd.Timestamp],
    entity_col: str = "store_id",
    run_name: Optional[str] = None,
) -> Dict:
    train_start, train_end, test_start, test_end = split

    df_train = df[(df["week_start"] >= train_start) & (df["week_start"] <= train_end)].copy()
    df_test  = df[(df["week_start"] >= test_start) & (df["week_start"] <= test_end)].copy()

    # Build design matrix based on panel method
    if panel_method == "fe":
        y_tr, X_tr, cols_used = within_transform(df_train, y_col, feature_cols, entity_col=entity_col)

    elif panel_method == "mundlak":
        df_aug, cols_used = add_mundlak_terms(df_train, feature_cols, entity_col=entity_col)
        y_tr = df_aug[y_col].values
        X_tr = df_aug[cols_used].values
    else:
        raise ValueError(f"Unknown panel_method: {panel_method}")

    # Fit
    if algorithm == "ols":
        fit = fit_ols_sm(y_tr, X_tr, cols_used)
    elif algorithm == "ridge":
        fit = fit_ridge(y_tr, X_tr, cols_used, alpha=float(algo_params["alpha"]))
    elif algorithm == "elasticnet":
        fit = fit_elasticnet(
            y_tr, X_tr, cols_used,
            alpha=float(algo_params["alpha"]),
            l1_ratio=float(algo_params["l1_ratio"])
        )
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")

    coef_table = compute_coef_table(fit)

    # MLflow logging
    with mlflow.start_run(run_name=run_name, nested=True):
        # Params (experiment axes)
        mlflow.log_param("target", y_col)
        mlflow.log_param("panel_method", panel_method)
        mlflow.log_param("algorithm", algorithm)
        mlflow.log_param("train_start", str(train_start.date()))
        mlflow.log_param("train_end", str(train_end.date()))
        mlflow.log_param("test_start", str(test_start.date()))
        mlflow.log_param("test_end", str(test_end.date()))
        mlflow.log_param("n_features", len(feature_cols))
        mlflow.log_param("n_obs_train", len(df_train))

        for k, v in algo_params.items():
            mlflow.log_param(f"hp_{k}", v)

        # Metrics (sanity only)
        for k, v in fit.metrics.items():
            mlflow.log_metric(k, v)

        # Artifacts: feature list + coefficient table
        mlflow.log_text("\n".join(feature_cols), artifact_file="features_used.txt")
        mlflow.log_text(json.dumps(algo_params, indent=2), artifact_file="algo_params.json")
        mlflow.log_text(coef_table.to_csv(index=False), artifact_file="coef_table.csv")

    # Standardized record for your survival aggregation later
    return {
        "target": y_col,
        "panel_method": panel_method,
        "algorithm": algorithm,
        "algo_params": algo_params,
        "train_start": train_start,
        "train_end": train_end,
        "test_start": test_start,
        "test_end": test_end,
        "coef_table": coef_table,     # keep in-memory for aggregation
        "fit_metrics": fit.metrics
    }


In [ ]:
def orchestrate_experiments(
    df: pd.DataFrame,
    targets: List[str],
    blocks: Dict[str, List[str]],
    panel_methods: List[str],
    algos: List[Tuple[str, Dict]],
    splits: List[Tuple[pd.Timestamp, pd.Timestamp, pd.Timestamp, pd.Timestamp]],
    block_sets: Optional[Dict[str, List[str]]] = None
):
    """
    block_sets lets you define feature subsets like:
      - single domain
      - domain combos (promo + pricing)
      - full model set
    """
    results = []
    if block_sets is None:
        # default: single-domain experiments
        block_sets = {k: v for k, v in blocks.items()}

    with mlflow.start_run(run_name="layer2_regression_experiments"):
        mlflow.log_param("n_splits", len(splits))
        mlflow.log_param("n_block_sets", len(block_sets))
        mlflow.log_param("targets", ",".join(targets))

        for target in targets:
            for block_name, feature_cols in block_sets.items():
                for panel_method in panel_methods:
                    for algo_name, algo_params in algos:
                        for s_idx, split in enumerate(splits):
                            run_name = f"{target}__{block_name}__{panel_method}__{algo_name}__split{s_idx:02d}"
                            out = run_one_experiment(
                                df=df,
                                y_col=target,
                                feature_cols=feature_cols,
                                panel_method=panel_method,
                                algorithm=algo_name,
                                algo_params=algo_params,
                                split=split,
                                run_name=run_name
                            )
                            out["block_name"] = block_name
                            out["split_idx"] = s_idx
                            results.append(out)

    return results


In [ ]:
blocks = infer_feature_blocks(df)

# targets: primary log-level first
targets = ["log_gc", "log_ac"]

panel_methods = ["fe", "mundlak"]

algos = [
    ("ols", {}),
    ("ridge", {"alpha": 1.0}),
    ("ridge", {"alpha": 10.0}),
    ("elasticnet", {"alpha": 0.1, "l1_ratio": 0.2}),
    ("elasticnet", {"alpha": 0.1, "l1_ratio": 0.8}),
]

splits = make_rolling_splits(df, train_weeks=104, test_weeks=13, step_weeks=13)

results = orchestrate_experiments(
    df=df,
    targets=targets,
    blocks=blocks,
    panel_methods=panel_methods,
    algos=algos,
    splits=splits
)


In [ ]:
def yoy_contributions_from_coef_table(
    df: pd.DataFrame,
    coef_table: pd.DataFrame,
    feature_prefix_filter: Optional[str] = None,
    week_t: Optional[pd.Timestamp] = None,
    entity_col: str = "store_id"
) -> pd.DataFrame:
    """
    Produces YoY contribution per feature at aggregate level:
      (mean x_t - mean x_t-52) * beta
    """
    ct = coef_table[["feature", "coef"]].copy()
    if feature_prefix_filter:
        ct = ct[ct["feature"].str.startswith(feature_prefix_filter)]

    if week_t is None:
        week_t = df["week_start"].max()
    week_tm52 = week_t - pd.Timedelta(weeks=52)

    df_t = df[df["week_start"] == week_t]
    df_b = df[df["week_start"] == week_tm52]

    rows = []
    for _, r in ct.iterrows():
        f = r["feature"]
        beta = r["coef"]
        if f not in df.columns:
            continue
        dx = df_t[f].mean() - df_b[f].mean()
        rows.append({
            "feature": f,
            "beta": beta,
            "dx_yoy": dx,
            "contrib_yoy": dx * beta
        })

    out = pd.DataFrame(rows).sort_values("contrib_yoy", ascending=False)
    out["abs_contrib_yoy"] = out["contrib_yoy"].abs()
    return out


In [ ]:
# Suppose you pick one GC run and one AC run (same feature set preferred)
gc_contrib = yoy_contributions_from_coef_table(df, gc_run["coef_table"], week_t=df["week_start"].max())
ac_contrib = yoy_contributions_from_coef_table(df, ac_run["coef_table"], week_t=df["week_start"].max())

sales_contrib = (
    gc_contrib[["feature", "contrib_yoy"]].rename(columns={"contrib_yoy":"gc_contrib_yoy"})
    .merge(ac_contrib[["feature", "contrib_yoy"]].rename(columns={"contrib_yoy":"ac_contrib_yoy"}),
           on="feature", how="outer")
    .fillna(0.0)
)
sales_contrib["sales_contrib_yoy"] = sales_contrib["gc_contrib_yoy"] + sales_contrib["ac_contrib_yoy"]
sales_contrib = sales_contrib.sort_values("sales_contrib_yoy", ascending=False)


In [ ]:
import pandas as pd
import numpy as np

def infer_domain_from_feature(feature: str) -> str:
    # Matches your synthetic prefixes; replace with your feature_catalog mapping if you have it
    for d in ["digital_promo", "nondigital_promo", "media", "menu", "mix_share", "csat", "pricing"]:
        if feature.startswith(d + "_"):
            return d
    return "other"

def results_to_long_table(results: list) -> pd.DataFrame:
    """
    results: list of outputs from run_one_experiment / orchestrate_experiments
    Returns: long table with one row per feature per run
    """
    rows = []
    for r in results:
        ct = r["coef_table"].copy()
        ct["target"] = r["target"]
        ct["block_name"] = r.get("block_name", None)
        ct["panel_method"] = r["panel_method"]
        ct["algorithm"] = r["algorithm"]
        ct["split_idx"] = r.get("split_idx", None)

        # For Mundlak runs: separate within X vs store-mean X if you want
        ct["is_mundlak_mean_term"] = ct["feature"].str.endswith("_mean_store")

        # Standard identifiers
        ct["run_key"] = (
            ct["target"].astype(str) + "|" +
            ct["block_name"].astype(str) + "|" +
            ct["panel_method"].astype(str) + "|" +
            ct["algorithm"].astype(str) + "|split" +
            ct["split_idx"].astype(str)
        )

        # Derived fields
        ct["domain"] = ct["feature"].map(infer_domain_from_feature)

        rows.append(ct)

    long_df = pd.concat(rows, ignore_index=True)

    # helpful flags
    long_df["has_pval"] = long_df["pval"].notna()
    long_df["sig_05"] = (long_df["pval"] < 0.05).astype(int)
    long_df["sig_10"] = (long_df["pval"] < 0.10).astype(int)

    # protect against tiny denom in CV
    long_df["abs_coef"] = long_df["coef"].abs()

    return long_df


In [ ]:
def compute_stability_metrics(
    long_df: pd.DataFrame,
    selected_only_for_stats: bool = True,
    significance_level: float = 0.05
) -> pd.DataFrame:
    """
    Returns one row per feature with stability metrics.
    """
    df = long_df.copy()

    # Optionally compute stats only where selected_flag==1
    if selected_only_for_stats:
        df_stats = df[df["selected_flag"] == 1].copy()
    else:
        df_stats = df.copy()

    sig_col = "sig_05" if significance_level == 0.05 else "sig_10"

    # Denominators
    total_runs_per_feature = df.groupby("feature")["run_key"].nunique().rename("n_runs_total")
    selected_runs_per_feature = df[df["selected_flag"] == 1].groupby("feature")["run_key"].nunique().rename("n_runs_selected")

    # Core stats (computed on df_stats)
    g = df_stats.groupby("feature")

    out = pd.DataFrame({
        "domain": g["domain"].first(),
        "n_rows_used_for_stats": g.size(),
        "coef_mean": g["coef"].mean(),
        "coef_median": g["coef"].median(),
        "coef_std": g["coef"].std(ddof=0),
        "abs_coef_mean": g["abs_coef"].mean(),
        "rank_abs_median": g["rank_abs"].median(),
        "sign_mode": g["sign"].agg(lambda s: int(np.sign(s.sum())) if len(s) else 0),
        "sign_consistency": g["sign"].agg(lambda s: np.nan if len(s) == 0 else (s.eq(s.mode().iloc[0]).mean())),
    }).reset_index()

    # CV = std/|mean|
    eps = 1e-12
    out["coef_cv"] = out["coef_std"] / (out["coef_mean"].abs() + eps)

    # Selection rates
    out = out.merge(total_runs_per_feature.reset_index(), on="feature", how="left")
    out = out.merge(selected_runs_per_feature.reset_index(), on="feature", how="left")
    out["n_runs_selected"] = out["n_runs_selected"].fillna(0).astype(int)
    out["pct_selected"] = out["n_runs_selected"] / out["n_runs_total"]

    # Sign consistency conditional on selected (recommended)
    # (If df_stats already selected-only, this is already conditional.)
    # For transparency:
    out["sign_consistency_cond_selected"] = out["sign_consistency"]

    # Significance: only on rows where p-values exist (OLS runs)
    df_sig_base = df_stats[df_stats["has_pval"]].copy()
    if not df_sig_base.empty:
        sig_rate = df_sig_base.groupby("feature")[sig_col].mean().rename(f"pct_significant_{int(significance_level*100)}")
        out = out.merge(sig_rate.reset_index(), on="feature", how="left")
    else:
        out[f"pct_significant_{int(significance_level*100)}"] = np.nan

    # Also helpful: number of distinct “worlds” (splits) survived
    survived_splits = df[df["selected_flag"] == 1].groupby("feature")["split_idx"].nunique().rename("n_splits_selected")
    out = out.merge(survived_splits.reset_index(), on="feature", how="left")
    out["n_splits_selected"] = out["n_splits_selected"].fillna(0).astype(int)

    return out


In [ ]:
def apply_selection_rules(
    stability_df: pd.DataFrame,
    min_pct_selected: float = 0.60,
    min_sign_consistency: float = 0.80,
    max_coef_cv: float = 1.00,
    max_median_rank: float = 20,
    min_pct_sig_5: float = 0.50,
    require_significance_if_available: bool = False
) -> pd.DataFrame:
    df = stability_df.copy()

    # Base filters
    keep = (
        (df["pct_selected"] >= min_pct_selected) &
        (df["sign_consistency_cond_selected"] >= min_sign_consistency) &
        (df["coef_cv"] <= max_coef_cv) &
        (df["rank_abs_median"] <= max_median_rank)
    )

    if require_significance_if_available:
        sig_col = "pct_significant_5"
        if sig_col in df.columns:
            keep = keep & (df[sig_col].isna() | (df[sig_col] >= min_pct_sig_5))

    df["pass_rules"] = keep.astype(int)

    # A useful composite score for sorting
    df["stability_score"] = (
        2.0 * df["pct_selected"].fillna(0) +
        1.5 * df["sign_consistency_cond_selected"].fillna(0) +
        1.0 * (1.0 / (1.0 + df["coef_cv"].fillna(10))) +
        0.5 * (1.0 / (1.0 + df["rank_abs_median"].fillna(999)))
    )

    return df.sort_values(["pass_rules", "stability_score"], ascending=[False, False])


In [ ]:
def enforce_domain_coverage(
    ranked_df: pd.DataFrame,
    final_k: int = 12,
    min_per_domain: dict = None,
    max_per_domain: dict = None
) -> pd.DataFrame:
    """
    ranked_df: output of apply_selection_rules() (already sorted)
    """
    if min_per_domain is None:
        min_per_domain = {
            "digital_promo": 1,
            "nondigital_promo": 1,
            "media": 1,
            "menu": 1,
            "pricing": 1,
            "csat": 1,
            # mix_share often capped rather than known-min
        }
    if max_per_domain is None:
        max_per_domain = {
            "mix_share": 2,
            "media": 2,
            "digital_promo": 2,
            "nondigital_promo": 2,
            "menu": 2,
            "pricing": 2,
            "csat": 2,
            "other": 1
        }

    chosen = []
    counts = {}

    # First: satisfy minimums using best-ranked per domain
    for dom, m in min_per_domain.items():
        cand = ranked_df[(ranked_df["domain"] == dom) & (ranked_df["pass_rules"] == 1)]
        for _, row in cand.head(m).iterrows():
            f = row["feature"]
            if f in chosen:
                continue
            chosen.append(f)
            counts[dom] = counts.get(dom, 0) + 1

    # Then: fill remaining slots by overall rank, respecting max caps
    for _, row in ranked_df.iterrows():
        if len(chosen) >= final_k:
            break
        if row["pass_rules"] != 1:
            continue
        dom = row["domain"]
        f = row["feature"]
        if f in chosen:
            continue
        if counts.get(dom, 0) >= max_per_domain.get(dom, 999):
            continue
        chosen.append(f)
        counts[dom] = counts.get(dom, 0) + 1

    out = ranked_df[ranked_df["feature"].isin(chosen)].copy()
    out["selected_final"] = 1
    # preserve chosen order
    out["selected_order"] = out["feature"].map({f:i for i,f in enumerate(chosen, start=1)})
    return out.sort_values("selected_order")


In [ ]:
# 1) build long table
long_df = results_to_long_table(results)

# (optional) exclude Mundlak mean terms if you only care about within X
# long_df = long_df[long_df["is_mundlak_mean_term"] == False].copy()

# 2) stability metrics
stab = compute_stability_metrics(long_df, selected_only_for_stats=True, significance_level=0.05)

# 3) apply rules + rank
ranked = apply_selection_rules(
    stab,
    min_pct_selected=0.60,
    min_sign_consistency=0.80,
    max_coef_cv=1.00,
    max_median_rank=20,
    require_significance_if_available=False
)

# 4) final driver list (8–15)
final = enforce_domain_coverage(ranked, final_k=12)

final[[
    "feature","domain","pct_selected","sign_consistency_cond_selected",
    "coef_mean","coef_cv","rank_abs_median","stability_score"
]].reset_index(drop=True)


In [ ]:
def pick_champion_run(results, target, block_name=None, panel_method="fe", algorithm="ridge", hp_filter=None):
    """
    Picks the run with the highest sanity metric (r2) among those matching the spec.
    hp_filter: dict like {"alpha": 10.0} to enforce specific hp
    """
    cand = []
    for r in results:
        if r["target"] != target:
            continue
        if block_name is not None and r.get("block_name") != block_name:
            continue
        if r["panel_method"] != panel_method:
            continue
        if r["algorithm"] != algorithm:
            continue
        if hp_filter:
            ok = True
            for k, v in hp_filter.items():
                if str(r["algo_params"].get(k)) != str(v):
                    ok = False
                    break
            if not ok:
                continue
        cand.append(r)
    if not cand:
        raise ValueError("No runs match the champion selection criteria.")
    # sanity metric only
    cand = sorted(cand, key=lambda x: x["fit_metrics"].get("r2", -np.inf), reverse=True)
    return cand[0]


In [ ]:
def fit_champion_on_full_window(
    df, y_col, feature_cols,
    panel_method="fe",
    algorithm="ridge",
    algo_params=None,
    entity_col="store_id",
    train_start="2023-01-02",
    train_end="2025-12-29"
):
    if algo_params is None:
        algo_params = {"alpha": 10.0} if algorithm == "ridge" else {}

    df_train = df[(df["week_start"] >= pd.Timestamp(train_start)) &
                  (df["week_start"] <= pd.Timestamp(train_end))].copy()

    if panel_method == "fe":
        y_tr, X_tr, cols_used = within_transform(df_train, y_col, feature_cols, entity_col=entity_col)
    elif panel_method == "mundlak":
        df_aug, cols_used = add_mundlak_terms(df_train, feature_cols, entity_col=entity_col)
        y_tr = df_aug[y_col].values
        X_tr = df_aug[cols_used].values
    else:
        raise ValueError(f"Unknown panel_method: {panel_method}")

    if algorithm == "ols":
        fit = fit_ols_sm(y_tr, X_tr, cols_used)
    elif algorithm == "ridge":
        fit = fit_ridge(y_tr, X_tr, cols_used, alpha=float(algo_params["alpha"]))
    elif algorithm == "elasticnet":
        fit = fit_elasticnet(y_tr, X_tr, cols_used,
                             alpha=float(algo_params["alpha"]),
                             l1_ratio=float(algo_params["l1_ratio"]))
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")

    coef_table = compute_coef_table(fit)
    return fit, coef_table


In [ ]:
def yoy_driver_contrib(
    df: pd.DataFrame,
    coef_table: pd.DataFrame,
    feature_cols: list,
    week_t: pd.Timestamp,
    yoy_lag_weeks: int = 52
) -> pd.DataFrame:
    """
    Returns YoY contribution per feature at aggregate level:
      (mean(x_t) - mean(x_t-lag)) * beta
    """
    week_tm = week_t - pd.Timedelta(weeks=yoy_lag_weeks)

    df_t = df[df["week_start"] == week_t]
    df_m = df[df["week_start"] == week_tm]

    beta = coef_table.set_index("feature")["coef"].reindex(feature_cols)

    rows = []
    for f in feature_cols:
        if f not in df.columns:
            continue
        dx = df_t[f].mean() - df_m[f].mean()
        b = float(beta.loc[f]) if pd.notna(beta.loc[f]) else 0.0
        rows.append({"feature": f, "dx_yoy": dx, "beta": b, "contrib_yoy": dx * b})

    out = pd.DataFrame(rows)
    out["abs_contrib_yoy"] = out["contrib_yoy"].abs()
    return out.sort_values("contrib_yoy", ascending=False)

def yoy_target_actual(df, log_target_col, week_t, lag_weeks=52):
    week_tm = week_t - pd.Timedelta(weeks=lag_weeks)
    y_t = df[df["week_start"] == week_t][log_target_col].mean()
    y_m = df[df["week_start"] == week_tm][log_target_col].mean()
    return float(y_t - y_m)


In [ ]:
def reconcile_sales_yoy(
    df: pd.DataFrame,
    week_t: pd.Timestamp,
    final_features: list,
    gc_coef_table: pd.DataFrame,
    ac_coef_table: pd.DataFrame
) -> dict:
    """
    Returns:
      - driver table (GC/AC/Sales contributions)
      - totals + residuals vs actual YoY deltas
    """
    gc = yoy_driver_contrib(df, gc_coef_table, final_features, week_t)
    ac = yoy_driver_contrib(df, ac_coef_table, final_features, week_t)

    driver = (
        gc[["feature", "contrib_yoy"]].rename(columns={"contrib_yoy": "gc_contrib_yoy"})
        .merge(ac[["feature", "contrib_yoy"]].rename(columns={"contrib_yoy": "ac_contrib_yoy"}),
               on="feature", how="outer")
        .fillna(0.0)
    )
    driver["sales_contrib_yoy"] = driver["gc_contrib_yoy"] + driver["ac_contrib_yoy"]
    driver["abs_sales_contrib_yoy"] = driver["sales_contrib_yoy"].abs()
    driver = driver.sort_values("sales_contrib_yoy", ascending=False)

    # Totals from model contributions
    total_gc = float(driver["gc_contrib_yoy"].sum())
    total_ac = float(driver["ac_contrib_yoy"].sum())
    total_sales_from_models = float(driver["sales_contrib_yoy"].sum())

    # Actual YoY deltas (aggregate)
    actual_gc = yoy_target_actual(df, "log_gc", week_t)
    actual_ac = yoy_target_actual(df, "log_ac", week_t)
    actual_sales = yoy_target_actual(df, "log_sales", week_t)

    # Identity checks
    identity_actual = actual_gc + actual_ac
    identity_model = total_gc + total_ac

    return {
        "driver_table": driver.reset_index(drop=True),
        "totals": {
            "week_t": str(week_t.date()),
            "actual_yoy_log_gc": actual_gc,
            "actual_yoy_log_ac": actual_ac,
            "actual_yoy_log_sales": actual_sales,
            "actual_identity_check_gc_plus_ac": identity_actual,

            "modeled_yoy_log_gc_from_drivers": total_gc,
            "modeled_yoy_log_ac_from_drivers": total_ac,
            "modeled_yoy_log_sales_from_drivers": total_sales_from_models,
            "modeled_identity_check_gc_plus_ac": identity_model,

            "residual_gc": actual_gc - total_gc,
            "residual_ac": actual_ac - total_ac,
            "residual_sales": actual_sales - total_sales_from_models,
        }
    }


In [ ]:
def run_full_reconciliation(
    df: pd.DataFrame,
    final_df: pd.DataFrame,
    week_t: pd.Timestamp = None,
    panel_method: str = "fe",
    algorithm: str = "ridge",
    algo_params_gc: dict = None,
    algo_params_ac: dict = None
):
    if week_t is None:
        week_t = df["week_start"].max()

    final_features = final_df["feature"].tolist()

    if algo_params_gc is None:
        algo_params_gc = {"alpha": 10.0} if algorithm == "ridge" else {}
    if algo_params_ac is None:
        algo_params_ac = {"alpha": 10.0} if algorithm == "ridge" else {}

    gc_fit, gc_coef = fit_champion_on_full_window(
        df, "log_gc", final_features,
        panel_method=panel_method, algorithm=algorithm, algo_params=algo_params_gc
    )
    ac_fit, ac_coef = fit_champion_on_full_window(
        df, "log_ac", final_features,
        panel_method=panel_method, algorithm=algorithm, algo_params=algo_params_ac
    )

    recon = reconcile_sales_yoy(
        df=df, week_t=week_t, final_features=final_features,
        gc_coef_table=gc_coef, ac_coef_table=ac_coef
    )

    return {
        "final_features": final_features,
        "gc_coef_table": gc_coef,
        "ac_coef_table": ac_coef,
        "driver_table": recon["driver_table"],
        "totals": recon["totals"]
    }


Baysian

import numpy as np
import pandas as pd

def build_bayes_design(
    df: pd.DataFrame,
    y_col: str,
    x_cols: list,
    store_col: str = "store_id",
    week_col: str = "week_start",
    domain_map: dict = None,   # feature -> domain string
):
    df = df.sort_values([store_col, week_col]).copy()

    # y, X
    y = df[y_col].values.astype(float)
    X = df[x_cols].values.astype(float)

    # store index 0..n_store-1
    store_codes, store_uniques = pd.factorize(df[store_col])
    store_idx = store_codes.astype(int)

    # week index (optional)
    week_codes, week_uniques = pd.factorize(df[week_col])
    week_idx = week_codes.astype(int)

    # domain index 0..n_domain-1 for each feature column
    if domain_map is None:
        # fallback: infer by prefix
        def infer_dom(f):
            for d in ["digital_promo","nondigital_promo","media","menu","mix_share","csat","pricing"]:
                if f.startswith(d + "_"):
                    return d
            return "other"
        domain_map = {f: infer_dom(f) for f in x_cols}

    domains = sorted(list(set(domain_map.values())))
    dom_to_id = {d:i for i,d in enumerate(domains)}
    feat_domain_idx = np.array([dom_to_id[domain_map[f]] for f in x_cols], dtype=int)

    return {
        "y": y,
        "X": X,
        "x_cols": x_cols,
        "store_idx": store_idx,
        "n_store": len(store_uniques),
        "week_idx": week_idx,
        "n_week": len(week_uniques),
        "feat_domain_idx": feat_domain_idx,
        "domains": domains
    }


In [ ]:
import pymc as pm
import arviz as az

def fit_hierarchical_bayes(
    design: dict,
    include_week_effect: bool = False,
    draws: int = 1000,
    tune: int = 1000,
    chains: int = 4,
    target_accept: float = 0.9,
    random_seed: int = 42,
):
    y = design["y"]
    X = design["X"]
    store_idx = design["store_idx"]
    n_store = design["n_store"]
    feat_dom = design["feat_domain_idx"]
    n_domain = len(design["domains"])

    if include_week_effect:
        week_idx = design["week_idx"]
        n_week = design["n_week"]

    with pm.Model() as model:
        # ---------- Random intercepts (store) ----------
        mu_a = pm.Normal("mu_a", mu=0.0, sigma=2.0)
        sigma_a = pm.HalfNormal("sigma_a", sigma=1.0)
        a_store = pm.Normal("a_store", mu=mu_a, sigma=sigma_a, shape=n_store)

        # ---------- Domain-level shrinkage for betas ----------
        # tau_d controls how large coefficients in domain d can be
        tau_domain = pm.HalfNormal("tau_domain", sigma=0.5, shape=n_domain)

        # coefficient per feature, variance depends on its domain
        beta = pm.Normal("beta", mu=0.0, sigma=tau_domain[feat_dom], shape=X.shape[1])

        # ---------- Optional week effect ----------
        if include_week_effect:
            sigma_w = pm.HalfNormal("sigma_w", sigma=0.5)
            w = pm.Normal("w", mu=0.0, sigma=sigma_w, shape=n_week)
            mu = a_store[store_idx] + w[week_idx] + pm.math.dot(X, beta)
        else:
            mu = a_store[store_idx] + pm.math.dot(X, beta)

        # ---------- Observation noise ----------
        sigma = pm.HalfNormal("sigma", sigma=1.0)
        y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

        # ---------- Sample ----------
        idata = pm.sample(
            draws=draws, tune=tune, chains=chains,
            target_accept=target_accept,
            random_seed=random_seed,
            return_inferencedata=True
        )

    return model, idata


In [ ]:
def posterior_feature_table(idata, x_cols, eps: float = 0.001, prob_threshold: float = 0.8) -> pd.DataFrame:
    """
    Returns a coefficient table similar to your frequentist outputs.
    Uses posterior to compute sign stability and selection probability.
    """
    beta_samples = idata.posterior["beta"].stack(sample=("chain","draw")).values  # shape: (p, n_samples)
    # PyMC/ArviZ may give shape (chain, draw, p) so we transpose if needed
    if beta_samples.shape[0] != len(x_cols):
        beta_samples = beta_samples.T

    means = beta_samples.mean(axis=1)
    sds = beta_samples.std(axis=1, ddof=0)

    # Probabilities
    p_pos = (beta_samples > 0).mean(axis=1)
    p_neg = (beta_samples < 0).mean(axis=1)
    p_nontrivial = (np.abs(beta_samples) > eps).mean(axis=1)

    # credible intervals
    q05 = np.quantile(beta_samples, 0.05, axis=1)
    q50 = np.quantile(beta_samples, 0.50, axis=1)
    q95 = np.quantile(beta_samples, 0.95, axis=1)

    dfc = pd.DataFrame({
        "feature": x_cols,
        "coef": means,
        "coef_sd": sds,
        "p_pos": p_pos,
        "p_neg": p_neg,
        "p_nontrivial": p_nontrivial,
        "ci05": q05,
        "ci50": q50,
        "ci95": q95,
    })
    dfc["sign"] = np.where(dfc["coef"] >= 0, 1, -1)
    dfc["abs_coef"] = dfc["coef"].abs()
    dfc["rank_abs"] = dfc["abs_coef"].rank(ascending=False, method="average")

    # Bayesian “selected” flag: nontrivial with high probability AND sign probability high
    dfc["sign_prob"] = dfc[["p_pos","p_neg"]].max(axis=1)
    dfc["selected_flag"] = ((dfc["p_nontrivial"] >= prob_threshold) & (dfc["sign_prob"] >= prob_threshold)).astype(int)

    # For compatibility with your frequentist schema (no p-values here)
    dfc["se"] = np.nan
    dfc["pval"] = np.nan

    return dfc.sort_values("abs_coef", ascending=False).reset_index(drop=True)


In [ ]:
def run_one_experiment_hb(
    df: pd.DataFrame,
    y_col: str,
    feature_cols: list,
    split: tuple,
    domain_map: dict = None,
    include_week_effect: bool = False,
    bayes_params: dict = None,
):
    if bayes_params is None:
        bayes_params = dict(draws=600, tune=600, chains=4, target_accept=0.9)

    train_start, train_end, test_start, test_end = split
    df_train = df[(df["week_start"] >= train_start) & (df["week_start"] <= train_end)].copy()

    design = build_bayes_design(
        df_train, y_col=y_col, x_cols=feature_cols,
        store_col="store_id", week_col="week_start",
        domain_map=domain_map
    )

    model, idata = fit_hierarchical_bayes(
        design,
        include_week_effect=include_week_effect,
        **bayes_params
    )

    coef_table = posterior_feature_table(idata, feature_cols, eps=0.001, prob_threshold=0.8)

    # sanity metrics (Bayes doesn't emphasize these, but you may log them)
    # Use posterior mean prediction for quick sanity:
    # (Optional, keep minimal for now)

    return {
        "target": y_col,
        "panel_method": "hb_domain_shrinkage" + ("_week" if include_week_effect else ""),
        "algorithm": "bayes",
        "algo_params": bayes_params,
        "train_start": train_start,
        "train_end": train_end,
        "test_start": test_start,
        "test_end": test_end,
        "coef_table": coef_table,
        "idata": idata,  # keep around if you want trace diagnostics
        "fit_metrics": {}  # optional
    }


In [ ]:
import mlflow
import json
import arviz as az

def log_hb_run_to_mlflow(run_out: dict):
    coef_table = run_out["coef_table"]
    idata = run_out.get("idata", None)

    with mlflow.start_run(run_name=run_out.get("run_name", None), nested=True):
        mlflow.log_param("target", run_out["target"])
        mlflow.log_param("panel_method", run_out["panel_method"])
        mlflow.log_param("algorithm", run_out["algorithm"])
        mlflow.log_param("train_start", str(run_out["train_start"].date()))
        mlflow.log_param("train_end", str(run_out["train_end"].date()))
        mlflow.log_param("test_start", str(run_out["test_start"].date()))
        mlflow.log_param("test_end", str(run_out["test_end"].date()))
        mlflow.log_param("n_features", coef_table.shape[0])

        mlflow.log_text(json.dumps(run_out["algo_params"], indent=2), "bayes_params.json")
        mlflow.log_text(coef_table.to_csv(index=False), "coef_table.csv")

        if idata is not None:
            summ = az.summary(idata, var_names=["beta"], round_to=6)
            # log a couple aggregate diagnostics
            mlflow.log_metric("beta_rhat_max", float(summ["r_hat"].max()))
            mlflow.log_metric("beta_ess_bulk_min", float(summ["ess_bulk"].min()))
            mlflow.log_text(summ.reset_index().to_csv(index=False), "posterior_summary_beta.csv")
